## Data Loader

In [170]:
%matplotlib inline

import os
import shutil
import random
import torch
import torchvision
import numpy as np
import time
import torch
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, random_split
import pdb
from PIL import Image
from matplotlib import pyplot as plt
import argparse
torch.manual_seed(0)

import torch.nn as nn
import torch.nn.functional as F

print('Using PyTorch version', torch.__version__)


from os.path import isfile
from os import rename
SMOOTH=1
import pdb
from sklearn.metrics import auc, roc_curve
from PIL.ImageFilter import GaussianBlur

# import pdb
from numpy import pi as PI
from numpy import sqrt
from scipy.special import comb
import math
# tensorflow libraries
# import tensorflow as tf
# from tensorflow import keras
# from keras.preprocessing.image import ImageDataGenerator

Using PyTorch version 1.11.0


In [171]:
# Miscellaneous initialization
torch.manual_seed(1)
start_time = time.time()
parser = argparse.ArgumentParser()

In [172]:
train_dataset_path = ""
test_dataset_path = ""

In [173]:
root_dir = "/Users/harshitb/Desktop/Studies/ECE 228/TB_dataset/TB_Chest_Radiography_Database"
class_names = ['Normal', 'Tuberculosis']
source_dirs = ['Normal', 'Tuberculosis']


In [174]:
parser.add_argument('--num_epochs', type=int, default=5, help='Number of training epochs')
parser.add_argument('--batch_size', type=int, default=32, help='Batch size')
parser.add_argument('--lr', type=float, default=5e-4, help='Learning rate')
parser.add_argument('--l2', type=float, default=0, help='L2 regularisation')
parser.add_argument('--aug', action='store_true', default=False, help='Use data augmentation')
parser.add_argument('--data_path', type=str, default=root_dir,help='Path to data.')
parser.add_argument('--bond_dim', type=int, default=5, help='MPS Bond dimension')
parser.add_argument('--nChannel', type=int, default=1, help='Number of input channels')
parser.add_argument('--dense_net', action='store_true', default=False, help='Using Dense Net model')

args = parser.parse_args([])

In [175]:
batch_size = args.batch_size

# LoTeNet parameters
adaptive_mode = False 
periodic_bc   = False

kernel = 2 # Stride along spatial dimensions
output_dim = 1 # output dimension
 
feature_dim = 2

#logFile = time.strftime("%Y%m%d_%H_%M")+'.txt'
#makeLogFile(logFile)

normTensor = 0.5*torch.ones(args.nChannel)
### Data processing and loading....


In [176]:
### Data processing and loading....
# mean=[0.485, 0.456, 0.406], std=[0.229,0.224,0.225]
train_transform = transforms.Compose([transforms.RandomHorizontalFlip(),
                                      transforms.Resize(size=(128,128)),
                                      transforms.RandomVerticalFlip(),
                                      transforms.RandomRotation(20),
                                      transforms.ToTensor(),
                                      transforms.Normalize(mean=normTensor,std=normTensor)])

valid_transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize(size=(128,128)),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomRotation(20),
    torchvision.transforms.ToTensor(),
    transforms.Normalize(mean=normTensor,std=normTensor)
    #torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229,0.224,0.225])
])

In [177]:
class ChestXRayDataset(torch.utils.data.Dataset):
    def __init__(self, image_dirs,transform):
        def get_images(class_name):
            images = [x for x in os.listdir(image_dirs[class_name]) if x.lower().endswith('png')]
            print(f'Found {len(images)}{class_name}')
            return images
        self.images={}
        self.class_names=['Normal', 'Tuberculosis']
        for c in self.class_names:
            self.images[c]=get_images(c)
        self.image_dirs=image_dirs
        self.transform=transform
        
    
    def __len__(self):
        return sum([len(self.images[c]) for c in self.class_names])
    def __getitem__(self, index):
        class_name=random.choice(self.class_names)
        index=index%len(self.images[class_name])
        image_name=self.images[class_name][index]
        image_path =os.path.join(self.image_dirs[class_name], image_name)
        image=Image.open(image_path).convert('L')
        return self.transform(image), self.class_names.index(class_name)

In [178]:
train_dirs = {
    'Normal': root_dir + '/Normal',
    'Tuberculosis': root_dir + '/Tuberculosis'
}
dataset = ChestXRayDataset(train_dirs, train_transform)

Found 3500Normal
Found 700Tuberculosis


In [179]:
f,l = dataset[0]

In [184]:
len(dataset)

4200

In [180]:
trainset, testset, valset = random_split(dataset, [3500, 500, 200])

In [181]:
train_loader =  torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
test_loader =  torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=True)

In [231]:
args.num_epochs = 5
batch_size

32

In [230]:
# num_epochs = args.num_epochs
# total_samples = 3500
# n_iterations = math.ceil(total_samples/batch_size)
# for epoch in range(args.num_epochs):
#     for i, (inputs, labels) in enumerate(train_loader):
#         if (i+1) % 10 == 0:
#             print(f'Epoch: {epoch+1}/{num_epochs}, \
#                     Step {i+1}/{n_iterations}| Inputs {inputs.shape} | Labels {labels.shape}')

In [215]:
x = inputs

conv1 = nn.Conv2d(1, 6, 5)
x = conv1(x)
print("C1: ", x.shape)

pool = nn.MaxPool2d(2, 2)
x = pool(x)
print("P1: ", x.shape)

conv2 = nn.Conv2d(6, 16, 5)
x = conv2(x)
print("C2: ", x.shape)

x = pool(x)
print("P1: ", x.shape)

xl = 1
for i in x.shape[1:]: xl *= i
fc1 = nn.Linear(xl, 120)
x = x.view(-1, xl) 
x = fc1(x)
print("F1: ", x.shape)

# self.fc1 = nn.Linear(16 * 5 * 5, 120)
# self.fc2 = nn.Linear(120, 84)
# self.fc3 = nn.Linear(84, 10)

C1:  torch.Size([12, 6, 124, 124])
P1:  torch.Size([12, 6, 62, 62])
C2:  torch.Size([12, 16, 58, 58])
P1:  torch.Size([12, 16, 29, 29])
F1:  torch.Size([12, 120])


In [216]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        #         xl = 1
        #         for i in x.shape[1:]: xl *= i
        self.fc1 = nn.Linear(13456, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2)

    def forward(self, x):
        # -> n, 3, 32, 32
        x = self.pool(F.relu(self.conv1(x)))  # -> n, 6, 14, 14
        x = self.pool(F.relu(self.conv2(x)))
        xl = 1
        for i in x.shape[1:]: xl *= i
        x = x.view(-1, xl)            # 
        x = F.relu(self.fc1(x))               # -> n, 120
        x = F.relu(self.fc2(x))               # -> n, 84
        x = self.fc3(x)                       # -> n, 2
        return x

In [233]:
model = ConvNet()
learning_rate = 0.001

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

n_total_steps = len(train_loader)
for epoch in range(args.num_epochs):
    for i, (images, labels) in enumerate(train_loader):

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 10 == 0:
            print (f'Epoch [{epoch+1}/{args.num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

Epoch [1/5], Step [10/110], Loss: 0.6940
Epoch [1/5], Step [20/110], Loss: 0.6943
Epoch [1/5], Step [30/110], Loss: 0.6946
Epoch [1/5], Step [40/110], Loss: 0.6938
Epoch [1/5], Step [50/110], Loss: 0.6906
Epoch [1/5], Step [60/110], Loss: 0.6952
Epoch [1/5], Step [70/110], Loss: 0.6935
Epoch [1/5], Step [80/110], Loss: 0.6953
Epoch [1/5], Step [90/110], Loss: 0.6905
Epoch [1/5], Step [100/110], Loss: 0.6957
Epoch [1/5], Step [110/110], Loss: 0.6946
Epoch [2/5], Step [10/110], Loss: 0.6901
Epoch [2/5], Step [20/110], Loss: 0.6949
Epoch [2/5], Step [30/110], Loss: 0.6873
Epoch [2/5], Step [40/110], Loss: 0.6905
Epoch [2/5], Step [50/110], Loss: 0.6944
Epoch [2/5], Step [60/110], Loss: 0.6934
Epoch [2/5], Step [70/110], Loss: 0.6908
Epoch [2/5], Step [80/110], Loss: 0.6910
Epoch [2/5], Step [90/110], Loss: 0.6915
Epoch [2/5], Step [100/110], Loss: 0.6918
Epoch [2/5], Step [110/110], Loss: 0.6925
Epoch [3/5], Step [10/110], Loss: 0.6933
Epoch [3/5], Step [20/110], Loss: 0.6951
Epoch [3/5],

In [ ]:
# pytorch res, res101.. 

In [234]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(2)]
    n_class_samples = [0 for i in range(2)]
    for images, labels in test_loader:
        #         print(len(labels))
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
        for i in range(len(labels)):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

    for i in range(2):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {class_names[i]}: {acc} %')

Accuracy of the network: 68.2 %
Accuracy of Normal: 36.554621848739494 %
Accuracy of Tuberculosis: 96.94656488549619 %
